In [1]:
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Dropout, Flatten, Input
from keras.optimizers import RMSprop
from keras.utils.np_utils import to_categorical
from PIL import Image
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import accuracy_score
from keras import backend as K

Using TensorFlow backend.


In [2]:
#training label
labels = np.load('labels.npy')
labels.shape

(33402, 62)

In [3]:
#img as train data
size = labels.shape[0]
numsample = 5
folder = 'train/croppedsampled/'
images = []

for k in range(numsample):
    for i in range(size):
        im = Image.open(folder+str(i+1)+'_'+str(k)+'.png')
        images.append(np.asarray(im))
        
images = np.array(images)
images.shape

(167010, 54, 54, 3)

In [4]:
#replicate labels to cover all sample
labels = np.tile(labels,(5,1))
labels.shape

(167010, 62)

In [5]:
#split training set and validation set
rs = ShuffleSplit(n_splits=1, test_size=.1, random_state=0)
train_index, test_index = next(rs.split(images)) #just 1
test_images = images[test_index]
test_labels = labels[test_index]
images = images[train_index]
labels = labels[train_index]

In [17]:
#initial model (same as initial model in other program)
a = Input(shape=(54, 54, 3))
#conv layer
c1 = Conv2D(64, 3,3, activation='relu', border_mode='same')(a)
c1p = MaxPooling2D(pool_size=(2, 2)) (c1)
c1d = Dropout(0.1)(c1p)

c2 = Conv2D(128, 3,3, activation='relu', border_mode='same')(c1d)
c2p = MaxPooling2D(pool_size=(2, 2)) (c2)
c2d = Dropout(0.1)(c2p)

c3 = Conv2D(192, 3,3, activation='relu', border_mode='same')(c2d)
c3p = MaxPooling2D(pool_size=(2, 2)) (c3)
c3d = Dropout(0.1)(c3p)


c4 = Conv2D(256, 3,3, activation='relu', border_mode='same')(c3d)
c4p = MaxPooling2D(pool_size=(2, 2)) (c4)
c4d = Dropout(0.1)(c4p)


c5 = Conv2D(256, 3,3, activation='relu', border_mode='same')(c4d)
c5p = MaxPooling2D(pool_size=(2, 2)) (c5)
c5d = Dropout(0.1)(c5p)

#linear layer
flat = Flatten()(c5d)
d1 = Dense(128, activation='relu')(flat)
d1d = Dropout(0.1)(d1)

#output
o_length = Dense(7, activation='softmax')(d1d)
o_digit1 = Dense(11, activation='softmax')(d1d)
o_digit2 = Dense(11, activation='softmax')(d1d)
o_digit3 = Dense(11, activation='softmax')(d1d)
o_digit4 = Dense(11, activation='softmax')(d1d)
o_digit5 = Dense(11, activation='softmax')(d1d)
model = Model(a, [o_length,o_digit1,o_digit2,o_digit3,o_digit4,o_digit5])

rms = RMSprop(lr=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=rms)


In [19]:
#training
result = model.fit(images, [labels[:,:7], labels[:,7:18], labels[:,18:29], labels[:,29:40], labels[:,40:51], labels[:,51:62]], 32, 20, validation_split=0.1, shuffle=True)

Train on 135278 samples, validate on 15031 samples
Epoch 1/20
135278/135278 [==============================] - 280s - loss: 5.3096 - dense_23_loss: 0.5698 - dense_24_loss: 1.6628 - dense_25_loss: 1.9013 - dense_26_loss: 0.9621 - dense_27_loss: 0.1930 - dense_28_loss: 0.0206 - val_loss: 3.6351 - val_dense_23_loss: 0.3500 - val_dense_24_loss: 1.0782 - val_dense_25_loss: 1.2932 - val_dense_26_loss: 0.7632 - val_dense_27_loss: 0.1490 - val_dense_28_loss: 0.0015
Epoch 2/20
135278/135278 [==============================] - 281s - loss: 3.1299 - dense_23_loss: 0.2591 - dense_24_loss: 0.9263 - dense_25_loss: 1.1297 - dense_26_loss: 0.6660 - dense_27_loss: 0.1453 - dense_28_loss: 0.0036 - val_loss: 2.3032 - val_dense_23_loss: 0.1819 - val_dense_24_loss: 0.6794 - val_dense_25_loss: 0.8002 - val_dense_26_loss: 0.5197 - val_dense_27_loss: 0.1202 - val_dense_28_loss: 0.0017
Epoch 3/20
135278/135278 [==============================] - 270s - loss: 2.3538 - dense_23_loss: 0.1981 - dense_24_loss: 0.6773

In [20]:
#validation
score = model.predict(test_images)

score = np.concatenate(score, axis=1)
#round score to get actual prediction instead of probability
roundedScore = np.zeros(score.shape, dtype="int32")
#choose threshold
roundedScore[score > 0.5] = 1

#rounded acc (exactly match)
accuracy_score(test_labels, roundedScore)

0.74929644931441231

In [6]:
#learning rate = 0.001 (instead of 0.0001 in initial model)
a = Input(shape=(54, 54, 3))
#conv layer
c1 = Conv2D(64, 3,3, activation='relu', border_mode='same')(a)
c1p = MaxPooling2D(pool_size=(2, 2)) (c1)
c1d = Dropout(0.1)(c1p)

c2 = Conv2D(128, 3,3, activation='relu', border_mode='same')(c1d)
c2p = MaxPooling2D(pool_size=(2, 2)) (c2)
c2d = Dropout(0.1)(c2p)

c3 = Conv2D(192, 3,3, activation='relu', border_mode='same')(c2d)
c3p = MaxPooling2D(pool_size=(2, 2)) (c3)
c3d = Dropout(0.1)(c3p)


c4 = Conv2D(256, 3,3, activation='relu', border_mode='same')(c3d)
c4p = MaxPooling2D(pool_size=(2, 2)) (c4)
c4d = Dropout(0.1)(c4p)


c5 = Conv2D(256, 3,3, activation='relu', border_mode='same')(c4d)
c5p = MaxPooling2D(pool_size=(2, 2)) (c5)
c5d = Dropout(0.1)(c5p)

#linear layer
flat = Flatten()(c5d)
d1 = Dense(128, activation='relu')(flat)
d1d = Dropout(0.1)(d1)

#output
o_length = Dense(7, activation='softmax')(d1d)
o_digit1 = Dense(11, activation='softmax')(d1d)
o_digit2 = Dense(11, activation='softmax')(d1d)
o_digit3 = Dense(11, activation='softmax')(d1d)
o_digit4 = Dense(11, activation='softmax')(d1d)
o_digit5 = Dense(11, activation='softmax')(d1d)
model = Model(a, [o_length,o_digit1,o_digit2,o_digit3,o_digit4,o_digit5])

rms = RMSprop(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=rms)


In [8]:
#training
result = model.fit(images, [labels[:,:7], labels[:,7:18], labels[:,18:29], labels[:,29:40], labels[:,40:51], labels[:,51:62]], 32, 10, validation_split=0.1, shuffle=True)

Train on 135278 samples, validate on 15031 samples
Epoch 1/10
135278/135278 [==============================] - 310s - loss: 4.7522 - dense_2_loss: 0.5240 - dense_3_loss: 1.4588 - dense_4_loss: 1.6471 - dense_5_loss: 0.9272 - dense_6_loss: 0.1853 - dense_7_loss: 0.0098 - val_loss: 2.9573 - val_dense_2_loss: 0.2653 - val_dense_3_loss: 0.9328 - val_dense_4_loss: 1.0257 - val_dense_5_loss: 0.6041 - val_dense_6_loss: 0.1272 - val_dense_7_loss: 0.0021
Epoch 2/10
135278/135278 [==============================] - 299s - loss: 3.3170 - dense_2_loss: 0.3253 - dense_3_loss: 0.9992 - dense_4_loss: 1.1527 - dense_5_loss: 0.6712 - dense_6_loss: 0.1642 - dense_7_loss: 0.0043 - val_loss: 2.8598 - val_dense_2_loss: 0.2616 - val_dense_3_loss: 0.9117 - val_dense_4_loss: 0.9863 - val_dense_5_loss: 0.5589 - val_dense_6_loss: 0.1392 - val_dense_7_loss: 0.0021
Epoch 3/10
135278/135278 [==============================] - 311s - loss: 3.3261 - dense_2_loss: 0.3363 - dense_3_loss: 1.0060 - dense_4_loss: 1.1424 - 

In [9]:
#validation
score = model.predict(test_images)

score = np.concatenate(score, axis=1)
#round score to get actual prediction instead of probability
roundedScore = np.zeros(score.shape, dtype="int32")
#choose threshold
roundedScore[score > 0.5] = 1

#rounded acc (exactly match)
accuracy_score(test_labels, roundedScore)

0.33045925393689002

In [9]:
#learning rate = 0.0005 (instead of 0.0001 in initial model)
a = Input(shape=(54, 54, 3))
#conv layer
c1 = Conv2D(64, 3,3, activation='relu', border_mode='same')(a)
c1p = MaxPooling2D(pool_size=(2, 2)) (c1)
c1d = Dropout(0.1)(c1p)

c2 = Conv2D(128, 3,3, activation='relu', border_mode='same')(c1d)
c2p = MaxPooling2D(pool_size=(2, 2)) (c2)
c2d = Dropout(0.1)(c2p)

c3 = Conv2D(192, 3,3, activation='relu', border_mode='same')(c2d)
c3p = MaxPooling2D(pool_size=(2, 2)) (c3)
c3d = Dropout(0.1)(c3p)


c4 = Conv2D(256, 3,3, activation='relu', border_mode='same')(c3d)
c4p = MaxPooling2D(pool_size=(2, 2)) (c4)
c4d = Dropout(0.1)(c4p)


c5 = Conv2D(256, 3,3, activation='relu', border_mode='same')(c4d)
c5p = MaxPooling2D(pool_size=(2, 2)) (c5)
c5d = Dropout(0.1)(c5p)

#linear layer
flat = Flatten()(c5d)
d1 = Dense(128, activation='relu')(flat)
d1d = Dropout(0.1)(d1)

#output
o_length = Dense(7, activation='softmax')(d1d)
o_digit1 = Dense(11, activation='softmax')(d1d)
o_digit2 = Dense(11, activation='softmax')(d1d)
o_digit3 = Dense(11, activation='softmax')(d1d)
o_digit4 = Dense(11, activation='softmax')(d1d)
o_digit5 = Dense(11, activation='softmax')(d1d)
model = Model(a, [o_length,o_digit1,o_digit2,o_digit3,o_digit4,o_digit5])

rms = RMSprop(lr=0.0005)
model.compile(loss='categorical_crossentropy', optimizer=rms)


In [11]:
#training
result = model.fit(images, [labels[:,:7], labels[:,7:18], labels[:,18:29], labels[:,29:40], labels[:,40:51], labels[:,51:62]], 32, 10, validation_split=0.1, shuffle=True)

Train on 135278 samples, validate on 15031 samples
Epoch 1/10
135278/135278 [==============================] - 287s - loss: 4.3449 - dense_9_loss: 0.5281 - dense_10_loss: 1.3172 - dense_11_loss: 1.5075 - dense_12_loss: 0.8045 - dense_13_loss: 0.1751 - dense_14_loss: 0.0125 - val_loss: 2.3975 - val_dense_9_loss: 0.2293 - val_dense_10_loss: 0.7184 - val_dense_11_loss: 0.8035 - val_dense_12_loss: 0.5244 - val_dense_13_loss: 0.1198 - val_dense_14_loss: 0.0021
Epoch 2/10
135278/135278 [==============================] - 277s - loss: 2.3714 - dense_9_loss: 0.2273 - dense_10_loss: 0.7010 - dense_11_loss: 0.7964 - dense_12_loss: 0.5017 - dense_13_loss: 0.1407 - dense_14_loss: 0.0043 - val_loss: 1.7284 - val_dense_9_loss: 0.1636 - val_dense_10_loss: 0.5074 - val_dense_11_loss: 0.5720 - val_dense_12_loss: 0.3715 - val_dense_13_loss: 0.1118 - val_dense_14_loss: 0.0021
Epoch 3/10
135278/135278 [==============================] - 286s - loss: 2.1431 - dense_9_loss: 0.2116 - dense_10_loss: 0.6337 - de

In [12]:
#validation
score = model.predict(test_images)

score = np.concatenate(score, axis=1)
#round score to get actual prediction instead of probability
roundedScore = np.zeros(score.shape, dtype="int32")
#choose threshold
roundedScore[score > 0.5] = 1

#rounded acc (exactly match)
accuracy_score(test_labels, roundedScore)

0.58768935991856774

In [6]:
#learning rate = 0.00005 (instead of 0.0001 in initial model)
a = Input(shape=(54, 54, 3))
#conv layer
c1 = Conv2D(64, 3,3, activation='relu', border_mode='same')(a)
c1p = MaxPooling2D(pool_size=(2, 2)) (c1)
c1d = Dropout(0.1)(c1p)

c2 = Conv2D(128, 3,3, activation='relu', border_mode='same')(c1d)
c2p = MaxPooling2D(pool_size=(2, 2)) (c2)
c2d = Dropout(0.1)(c2p)

c3 = Conv2D(192, 3,3, activation='relu', border_mode='same')(c2d)
c3p = MaxPooling2D(pool_size=(2, 2)) (c3)
c3d = Dropout(0.1)(c3p)


c4 = Conv2D(256, 3,3, activation='relu', border_mode='same')(c3d)
c4p = MaxPooling2D(pool_size=(2, 2)) (c4)
c4d = Dropout(0.1)(c4p)


c5 = Conv2D(256, 3,3, activation='relu', border_mode='same')(c4d)
c5p = MaxPooling2D(pool_size=(2, 2)) (c5)
c5d = Dropout(0.1)(c5p)

#linear layer
flat = Flatten()(c5d)
d1 = Dense(128, activation='relu')(flat)
d1d = Dropout(0.1)(d1)

#output
o_length = Dense(7, activation='softmax')(d1d)
o_digit1 = Dense(11, activation='softmax')(d1d)
o_digit2 = Dense(11, activation='softmax')(d1d)
o_digit3 = Dense(11, activation='softmax')(d1d)
o_digit4 = Dense(11, activation='softmax')(d1d)
o_digit5 = Dense(11, activation='softmax')(d1d)
model = Model(a, [o_length,o_digit1,o_digit2,o_digit3,o_digit4,o_digit5])

rms = RMSprop(lr=0.00005)
model.compile(loss='categorical_crossentropy', optimizer=rms)


In [7]:
#training
result = model.fit(images, [labels[:,:7], labels[:,7:18], labels[:,18:29], labels[:,29:40], labels[:,40:51], labels[:,51:62]], 32, 20, validation_split=0.1, shuffle=True)

Train on 135278 samples, validate on 15031 samples
Epoch 1/20
135278/135278 [==============================] - 296s - loss: 6.6242 - dense_2_loss: 0.8377 - dense_3_loss: 2.0854 - dense_4_loss: 2.2692 - dense_5_loss: 1.1439 - dense_6_loss: 0.2586 - dense_7_loss: 0.0295 - val_loss: 5.3667 - val_dense_2_loss: 0.5023 - val_dense_3_loss: 1.7528 - val_dense_4_loss: 1.9422 - val_dense_5_loss: 0.9543 - val_dense_6_loss: 0.2091 - val_dense_7_loss: 0.0059
Epoch 2/20
135278/135278 [==============================] - 278s - loss: 4.6692 - dense_2_loss: 0.4085 - dense_3_loss: 1.4931 - dense_4_loss: 1.7441 - dense_5_loss: 0.8542 - dense_6_loss: 0.1658 - dense_7_loss: 0.0034 - val_loss: 3.6948 - val_dense_2_loss: 0.3065 - val_dense_3_loss: 1.1494 - val_dense_4_loss: 1.3782 - val_dense_5_loss: 0.7210 - val_dense_6_loss: 0.1385 - val_dense_7_loss: 0.0011
Epoch 3/20
135278/135278 [==============================] - 277s - loss: 3.5843 - dense_2_loss: 0.2928 - dense_3_loss: 1.0944 - dense_4_loss: 1.3205 - 

135278/135278 [==============================] - 289s - loss: 1.1282 - dense_2_loss: 0.0929 - dense_3_loss: 0.3272 - dense_4_loss: 0.3650 - dense_5_loss: 0.2473 - dense_6_loss: 0.0917 - dense_7_loss: 0.0042 - val_loss: 0.9638 - val_dense_2_loss: 0.0862 - val_dense_3_loss: 0.2812 - val_dense_4_loss: 0.3032 - val_dense_5_loss: 0.2116 - val_dense_6_loss: 0.0796 - val_dense_7_loss: 0.0021ss: 0.0910 - dense_7_loss: 1.5 - ETA: 271s - loss: 1.1062 -  - ETA: 261s - loss: 1.1377 - dense_2_loss: 0.0871 - dense_3_loss: 0.3293 - dense_4_loss: 0.3763 - dense_5_loss: 0.2477 - d - ETA: 258s - loss: 1.1429 - dense_2_loss: 0.0893 - dense_3_loss: 0.3330  - ETA: 252s - loss: 1.1470 - dense_2_loss: 0.0909 - dense_3_loss: 0.3358 - dense_4_loss: 0.3749 - dense_5_loss: 0.246 - ETA: 249s - loss: 1.1417 - dense_2_loss: 0.0887 - dense_3_loss: 0.3361 - dense_4_loss: 0.3719 - dense_5_loss: 0.2459 - dense_6_loss: 0.093 - ETA: 247s - loss: 1.1430 - dense_2_loss: 0.0888 - dense_3_loss: 0.332 - ETA: 241s - loss: 1.13

135278/135278 [==============================] - 286s - loss: 1.0685 - dense_2_loss: 0.0899 - dense_3_loss: 0.3128 - dense_4_loss: 0.3402 - dense_5_loss: 0.2330 - dense_6_loss: 0.0884 - dense_7_loss: 0.0042 - val_loss: 0.9139 - val_dense_2_loss: 0.0823 - val_dense_3_loss: 0.2685 - val_dense_4_loss: 0.2851 - val_dense_5_loss: 0.2000 - val_dense_6_loss: 0.0757 - val_dense_7_loss: 0.0021_loss: 0.0864 - dense_3_loss: 0.3138 - dense_4_loss: 0.3445 - dense_5_loss: 0.2339 - dense_6_loss: 0. - ETA: 258s - loss: 1.0809 - dense_2_loss: 0.0881 - dense_3_loss: 0.3185 - dense_4_loss: 0.3450 - dense_ - ETA: 254s - loss: 1.0788 - dense_2_loss: 0.0895 - dense_3_loss: 0.3176 - dense_4_loss: 0.3443 - dense_5_loss: 0. - ETA: 251s - loss: 1. - ETA: 242s - loss: 1.0587 - dense_2_loss: 0.0884 - dense_3_loss: 0.3109 - dense_4_loss: 0.3348 - dense_5_loss: 0.2335 - dense_6_loss: 0.0869 - d - ETA: 240s - loss: 1.0546 - dense_2_loss: 0.0882 - dense_3_loss: 0.3110 - dense_4_loss: 0.3321 - dense_5_loss: 0.2332 - d

135278/135278 [==============================] - 292s - loss: 1.0369 - dense_2_loss: 0.0848 - dense_3_loss: 0.3016 - dense_4_loss: 0.3329 - dense_5_loss: 0.2271 - dense_6_loss: 0.0862 - dense_7_loss: 0.0042 - val_loss: 0.9194 - val_dense_2_loss: 0.0922 - val_dense_3_loss: 0.2638 - val_dense_4_loss: 0.2803 - val_dense_5_loss: 0.2035 - val_dense_6_loss: 0.0773 - val_dense_7_loss: 0.00210.3432 - dense_4_loss: 0.3069 - dense_5_loss - ETA: 273s - loss: 1.0161 - dense_2_loss: 0.0861 - dense_3_loss: 0.3080 - ETA: 267s - loss: 1.0197 - dense_2_loss: 0.0851 - dense_3_loss: 0.3027 - dense_4_loss: 0.3159 - dense_5_loss: 0.2329 - dense_6 - ETA: 264s - loss: 1.0002 - dense_2_loss: 0.0830 - dense_3_loss: 0.2987 - dense_4_loss: 0. - ETA: 259s - loss: 1.0157 - dense_2_loss: 0.0877 - dense_3_loss: 0.2980 - dense_4_loss: 0.3162 - dense_5_loss: 0.2304 - dense_6_loss: 0.0789 - dense_7_loss - ETA: 259s - loss: 1.0137 - dense_2_loss: 0.0876 - dense_3_loss: 0.2964 - dense_4_loss: 0.3158 - dense_5_loss: 0.229

In [8]:
#validation
score = model.predict(test_images)

score = np.concatenate(score, axis=1)
#round score to get actual prediction instead of probability
roundedScore = np.zeros(score.shape, dtype="int32")
#choose threshold
roundedScore[score > 0.5] = 1

#rounded acc (exactly match)
accuracy_score(test_labels, roundedScore)

0.75839770073648283